## Produce Images from Igor Binary Wave

### Import necessary packages 
define useful function

In [36]:
%matplotlib inline
import os, sys
from ipywidgets import interact
try:
    import h5py
except ImportError:
    !python -m pip install h5py
try:
    from tqdm.notebook import tqdm
except ImportError:
    !python -m pip install tqdm
try:
    from pyprnt import prnt
except ImportError:
    !python -m pip install tqdm
try:
    from matplotlib import pyplot as plt 
except ImportError:
    !python -m pip install matplotlib
try:
    import ibw_utils as ibw
except ImportError:
    print('Check the ibw_utils.py file')
    
def mkfolder(path):
    if not os.path.exists(path):
        os.mkdir(path)

### Set the main path or folder to generate images
==> Change the name of dataset under data folder

In [37]:
dataset = 'Stress_pvdf'
path = os.path.join('./data', dataset)
h5Files = []
print('Loading the ibw files...')
for filename in tqdm([f for f in os.listdir(path) if f[-3:] == 'ibw']):
    if os.path.isfile(os.path.join(path, filename[:-3]+'h5')):
        h5 = h5py.File(os.path.join(path, filename[:-3]+'h5'), 'r')
        h5 = h5['Measurement_000/']
    else:
        h5 = ibw.ibw2h5(os.path.join(path, filename))
    ImageObj = ibw.AngleImage(filename[:-4], h5)
    h5Files.append(ImageObj)
h5Files = sorted(h5Files, key=lambda h5: h5.namedangle)

Loading the ibw files...


These lines will show the list of channels in the loaded ibw files

In [38]:

TypeNum, channels = ibw.channel_list_folder(h5Files)
if TypeNum == 1:
    print('\nChannel List of {} Folder'.format(dataset))
    prnt(channels)
else:
    type_dict, type_files, channel_type = channels
    print('\nThere are multiple types of files in the folder...')
    for key, val in type_dict.items():
        print('Type {}'.format(key))
        print('{} files in this type:\n{}'.format(len(val.split(',')), val))
        print('Channel List of Type {}'.format(key))
        prnt(channel_type[type_files[key-1]])


Channel List of Stress_pvdf Folder
┌───────────┬───────────────────┐
│Channel_000│HeightTrace        │
│Channel_001│HeightRetrace      │
│Channel_002│AmplitudeTrace     │
│Channel_003│AmplitudeRetrace   │
│Channel_004│DeflectionTrace    │
│Channel_005│DeflectionRetrace  │
│Channel_006│PhaseTrace         │
│Channel_007│PhaseRetrace       │
│Channel_008│UserCalcTrace      │
│Channel_009│UserCalcRetrace    │
│Channel_010│UserCalcBTrace     │
│Channel_011│UserCalcBRetrace   │
│Channel_012│LatAmplitudeTrace  │
│Channel_013│LatAmplitudeRetrace│
│Channel_014│LatPhaseTrace      │
│Channel_015│LatPhaseRetrace    │
│Channel_016│HeightRetrace      │
└───────────┴───────────────────┘


Set the channel to convert images, Images are saved in 'data/dataset/Images/Channel(Flatten)'

In [39]:
channel = 1

ChannelName = channels['Channel_'+str(channel).zfill(3)]
print('Selected channel name: ', ChannelName)

ImagePath = os.path.join(path, 'Images')
if flatten:
    ChannelPath = os.path.join(ImagePath, ChannelName+'_Flatten')
else:
    ChannelPath = os.path.join(ImagePath, ChannelName)    

cmaplist = ['viridis', 'plasma', 'inferno', 'magma', 'cividis', 
            'Greys', 'Purples', 'Blues', 'Greens', 'Oranges', 'Reds',
            'YlOrBr', 'YlOrRd', 'OrRd', 'PuRd', 'RdPu', 'BuPu',
            'GnBu', 'PuBu', 'YlGnBu', 'PuBuGn', 'BuGn', 'YlGn',
            'binary', 'gist_yarg', 'gist_gray', 'gray', 'bone', 'pink',
            'spring', 'summer', 'autumn', 'winter', 'cool', 'Wistia',
            'hot', 'afmhot', 'gist_heat', 'copper', 'PiYG', 'PRGn',
            'BrBG', 'PuOr', 'RdGy', 'RdBu', 'RdYlBu', 'RdYlGn', 'Spectral',
            'coolwarm', 'bwr', 'seismic', 'twilight', 'twilight_shifted', 'hsv',
            'Pastel1', 'Pastel2', 'Paired', 'Accent', 'Dark2', 'Set1', 'Set2',
            'Set3', 'tab10', 'tab20', 'tab20b', 'tab20c', 'flag', 'prism',
            'ocean', 'gist_earth', 'terrain', 'gist_stern',
            'gnuplot', 'gnuplot2', 'CMRmap', 'cubehelix', 'brg',
            'gist_rainbow', 'rainbow', 'jet', 'nipy_spectral', 'gist_ncar']
cmapdict = {}
for item in cmaplist:
    cmapdict[item] = item

Selected channel name:  HeightRetrace


In [47]:
@interact(color = cmaplist, flattenorder = (-1, 5, 1))
def DisplayImage(color, flattenorder=-1):
    if flattenorder == -1:
        img, cmap_img = ibw.h5toimg(h5Files[0].data, str(channel))
    else:
        img, cmap_img = ibw.h5toimg(h5Files[0].data, str(channel))
        img = ibw.ImageFlatten(img, flattenorder)
    f, axs = plt.subplots(1, 2, figsize=(15, 15))
    plt.subplot(121)
    plt.imshow(img, cmap=cmap_img)
    plt.subplot(122)
    plt.imshow(img, cmap=color)
    plt.show()

interactive(children=(Dropdown(description='color', options=('viridis', 'plasma', 'inferno', 'magma', 'cividis…

In [49]:
color = 'RdPu'
flatten = True 
flatten_order = 1
mkfolder(ImagePath)
mkfolder(ChannelPath)

for idx in range(len(h5Files)):
    img, cmap_img = ibw.h5toimg(h5Files[idx].data, str(channel))
    if not cmap in cmaplist:
        color = cmap_img
    if flatten:
        img = ibw.ImageFlatten(img, int(flatten_order))
    plt.imsave(os.path.join(ChannelPath, h5Files[idx].imagename+'.png'), img, cmap=color)
    print(h5Files[idx].imagename+'.png', 'is saved!')

VectorPFM_High_V0053_-90.png is saved!
VectorPFM_High_V0047_-75.png is saved!
VectorPFM_High_V0044_-60.png is saved!
VectorPFM_High_V0039_-45.png is saved!
VectorPFM_High_V0036_-30.png is saved!
VectorPFM_High_V0031_-15.png is saved!
VectorPFM_High_V0001_0.png is saved!
VectorPFM_High_V0005_+15.png is saved!
VectorPFM_High_V0010_+30.png is saved!
VectorPFM_High_V0014_+45.png is saved!
VectorPFM_High_V0019_+60.png is saved!
VectorPFM_High_V0023_+75.png is saved!
VectorPFM_High_V0028_+90.png is saved!
